# generation_projects_info, graph_tech_colors, graph_tech_types, non_fuel_energy_sources

In [1]:
'''
Import based on imports found in the notebooks located here:
Schivley Greg, PowerGenome, (2022), GitHub repository, 
    https://github.com/PowerGenome/PowerGenome/tree/master/notebooks
'''
import pandas as pd
import numpy as np

###
# module issue
import os
import sys
module_path = os.path.abspath(os.getcwd() + '\\..')
if module_path not in sys.path:
    sys.path.append(module_path)
###
from pathlib import Path
from powergenome.util import (
    build_scenario_settings,
    init_pudl_connection,
    load_settings,
    check_settings
)

cwd = Path.cwd()

settings_path = (cwd / "settings_TD.yml")
settings = load_settings(settings_path)

In [2]:
# read in csv outputs from PG
existing_gen = pd.read_csv('PG_output_csv/existing_gen_WECC.csv', index_col=0)
# new_gen = pd.read_csv('PG_output_csv/new_gen_WECC.csv', index_col=0)
# existing_variability = pd.read_csv('PG_output_csv/existing_variability_WECC.csv', index_col=0)
potential_build_yr = pd.read_csv('PG_output_csv/gc_units_model_WECC.csv', index_col=0)
all_gen = pd.read_csv('PG_output_csv/all_gen_WECC.csv', index_col=0)
# fuels = pd.read_csv('PG_output_csv/fuels_WECC.csv', index_col=0)
fuel_prices = pd.read_csv('PG_output_csv/fuel_prices_WECC.csv', index_col=0)
# load_curves = pd.read_csv('PG_output_csv/load_curves_WECC.csv', index_col=0)

In [3]:
# assumed cogen to be false
# based on REAM
cogen_tech = {'Onshore Wind Turbine': False, 'Biomass': False, 'Conventional Hydroelectric': False, 
             'Conventional Steam Coal': False, 'Natural Gas Fired Combined Cycle': False, 
             'Natural Gas Fired Combustion Turbine': False, 'Natural Gas Steam Turbine': False, 
             'Nuclear': False, 'Solar Photovoltaic': False, 'Hydroelectric Pumped Storage': False, 
             'Offshore Wind Turbine': False, 'Small Hydroelectric': False, 
              'NaturalGas_CCCCSAvgCF_Conservative': False, 'NaturalGas_CCAvgCF_Moderate': False, 
             'NaturalGas_CTAvgCF_Moderate': False, 'Battery_*_Moderate': False, 
              'NaturalGas_CCS100_Moderate': False, 'heat_load_shifting': False}

# based on REAM
baseload_tech = {'Onshore Wind Turbine': False, 'Biomass': False, 'Conventional Hydroelectric': False, 
                'Conventional Steam Coal': True, 'Natural Gas Fired Combined Cycle': False, 
                'Natural Gas Fired Combustion Turbine': False, 'Natural Gas Steam Turbine': False,
                'Nuclear': True, 'Solar Photovoltaic': False, 'Hydroelectric Pumped Storage': False, 
                'Offshore Wind Turbine': False, 'Small Hydroelectric': False, 
                'NaturalGas_CCCCSAvgCF_Conservative': False, 'NaturalGas_CCAvgCF_Moderate': False, 
                'NaturalGas_CTAvgCF_Moderate': False, 'Battery_*_Moderate': False, 
                'NaturalGas_CCS100_Moderate': False, 'heat_load_shifting': False}

# based on technology name
energy_tech = {'Onshore Wind Turbine': 'Wind', 'Biomass': 'Bio Solid', 'Conventional Hydroelectric': 'Water', 
                'Conventional Steam Coal': 'Coal', 'Natural Gas Fired Combined Cycle': 'Gas', 
                'Natural Gas Fired Combustion Turbine': 'Gas', 'Natural Gas Steam Turbine': 'Gas',
                'Nuclear': 'Uranium', 'Solar Photovoltaic': 'Solar', 'Hydroelectric Pumped Storage': 'Water', 
                'Offshore Wind Turbine': 'Wind', 'Small Hydroelectric': 'Water', 
                'NaturalGas_CCCCSAvgCF_Conservative': 'Gas', 'NaturalGas_CCAvgCF_Moderate': 'Gas', 
                'NaturalGas_CTAvgCF_Moderate': 'Gas', 'Battery_*_Moderate': 'Electricity', 
                'NaturalGas_CCS100_Moderate': 'Gas', 'heat_load_shifting': False}

# outage rates based on technology average value from REAM scenario 178 (ignored those that were cogen)
sched_outage_tech = {'Onshore Wind Turbine': 0.0, 'Biomass': 0.06, 'Conventional Hydroelectric': 0.05, 
                'Conventional Steam Coal': 0.06, 'Natural Gas Fired Combined Cycle': 0.6, 
                'Natural Gas Fired Combustion Turbine': 0.6, 'Natural Gas Steam Turbine': 0.6,
                'Nuclear': 0.06, 'Solar Photovoltaic': 0.0, 'Hydroelectric Pumped Storage': 0.05, 
                'Offshore Wind Turbine': 0.01, 'Small Hydroelectric': 0.05, 
                'NaturalGas_CCCCSAvgCF_Conservative': 0.6, 'NaturalGas_CCAvgCF_Moderate': 0.6, 
                'NaturalGas_CTAvgCF_Moderate': 0.6, 'Battery_*_Moderate': 0.01, 
                'NaturalGas_CCS100_Moderate': 0.6, 'heat_load_shifting': False}
forced_outage_tech = {'Onshore Wind Turbine': 0.0, 'Biomass': 0.04, 'Conventional Hydroelectric': 0.05, 
                'Conventional Steam Coal': 0.04, 'Natural Gas Fired Combined Cycle': 0.4, 
                'Natural Gas Fired Combustion Turbine': 0.4, 'Natural Gas Steam Turbine': 0.4,
                'Nuclear': 0.04, 'Solar Photovoltaic': 0.0, 'Hydroelectric Pumped Storage': 0.05, 
                'Offshore Wind Turbine': 0.05, 'Small Hydroelectric': 0.05, 
                'NaturalGas_CCCCSAvgCF_Conservative': 0.4, 'NaturalGas_CCAvgCF_Moderate': 0.4, 
                'NaturalGas_CTAvgCF_Moderate': 0.4, 'Battery_*_Moderate': 0.02, 
                'NaturalGas_CCS100_Moderate': 0.4, 'heat_load_shifting': False}
# take out heat_load_shifting - not in SWITCH

In [4]:
# to help calculate gen_connect_cost_per_mw
spur_capex_mw_mile = settings.get('transmission_investment_cost')['spur']['capex_mw_mile']

# to populate gen_max_age
retirement_age = settings.get('retirement_ages')
# add missing keys, values based on https://www.nrel.gov/docs/fy22osti/80641.pdf
retirement_age['Biomass'] = 50
retirement_age['NaturalGas_CCCCSAvgCF_Conservative'] = 60 # combined cycle and carbon capture sequestration
retirement_age['NaturalGas_CCAvgCF_Moderate'] = 60 # carbon capture
retirement_age['NaturalGas_CTAvgCF_Moderate'] = 50 # combustion turbine
retirement_age['Battery_*_Moderate'] = 15
retirement_age['NaturalGas_CCS100_Moderate'] = 60
retirement_age['heat_load_shifting'] = 10 # deleting

In [5]:
def generation_projects_info(all_gen, spur_capex_mw_mile, retirement_age, cogen_tech, baseload_tech, 
                            energy_tech, sched_outage_tech, forced_outage_tech):
    '''
    Create the generation_projects_info table based on REAM scenario 178.
    Inputs:
        * all_gen: from PowerGenome gc.create_all_generators()
        * spur_capex_mw_mile: based on the settings file ('transmission_investment_cost')['spur']['capex_mw_mile']
        * retirement age: pulled from settings
        * cogen_tech, baseload_tech, energy_tech, sched_outage_tech, forced_outage_tech
            - these are user defined dictionaries.  Will map values based on the technology
    Output columns:
        * GENERATION_PROJECT: basing on index
        * gen_tech: based on technology
        * gen_energy_source: based on energy_tech input
        * gen_load_zone: IPM region
        * gen_max_age: based on retirement_age
        * gen_is_variable: only solar and wind are true
        * gen_is_baseload: based on baseload_tech
        * gen_full_load_heat_rate: based on Heat_Rate_MMBTU_per_MWh from all_gen
            - if the energy_source is in the non_fuel_energy_sources, this should be '.'
        * gen_variable_om: based on var_om_cost_per_MWh from all_gen
        * gen_connect_cost_per_mw: based on spur_capex_mw_mile * spur_miles
        * gen_dbid: same as generation_project
        * gen_scheduled_outage_rate: based on sched_outage_tech
        * gen_forced_outage_rate: based on forced_outage_tech
        * gen_capacity_limit_mw: based on Existing_Cap_MW from all_gen
        * gen_min_build_capacity: based on REAM using 0 for now
        * gen_is_cogen: based on cogen_tech input
        * gen_storage_efficiency: based on REAM scenario 178.  batteries use 0.75
        * gen_store_to_release_ratio: based on REAM scenario 178. batteries use 1
        * gen_can_provide_cap_reserves: based on REAM, all 1s
        * gen_self_discharge_rate, gen_discharge_efficiency, gen_land_use_rate, gen_storage_energy_to_power_ratio: 
            blanks based on REAM
    '''

    gen_project_info = all_gen.copy()

    # get columns for GENERATION_PROJECT, gen_tech, gen_load_zone, gen_full_load_heat_rate, gen_variable_om, 
    # gen_connect_cost_per_mw and gen_capacity_limit_mw
    gen_project_info = gen_project_info[['index', 'technology', 'region', 'Heat_Rate_MMBTU_per_MWh', 
                                         'Var_OM_Cost_per_MWh', 'spur_miles', 'Existing_Cap_MW']]

    # create gen_connect_cost_per_mw from spur_miles and spur_capex_mw_mile
    gen_project_info['spur_capex_mw_mi'] = gen_project_info['region'].apply(lambda x: spur_capex_mw_mile[x])
    gen_project_info['spur_miles'] = gen_project_info['spur_miles'].fillna(0)
    gen_project_info['gen_connect_cost_per_mw'] = gen_project_info['spur_capex_mw_mi'] * gen_project_info['spur_miles']
    gen_project_info = gen_project_info.drop(['spur_miles', 'spur_capex_mw_mi'], axis=1)

    # Heat_Rate_MMBTU_per_MWh needs to be converted to Btu/kWh for gen_full_load_heat_rate
    # mmbtu * 1000000 = btu and 1 mwh * 1000 = kwh
    # 1000000 / 1000 = * 1000
    gen_project_info['Heat_Rate_MMBTU_per_MWh'] = gen_project_info['Heat_Rate_MMBTU_per_MWh'].apply(lambda x: 
                                                                                                    x*1000)

    # for gen_is_variable - only solar and wind technologies are true
    technology = all_gen['technology'].to_list()
    def Filter(list1, list2):
        return [n for n in list1 if
                 any(m in n for m in list2)]
    wind_solar = set(Filter(technology, ['Wind', 'Solar']))
    gen_project_info.loc[gen_project_info['technology'].isin(wind_solar), 'gen_is_variable'] = True
    gen_project_info['gen_is_variable'] = gen_project_info['gen_is_variable'].fillna(False)

    # gen_storage_efficiency and gen_store_to_release_ratio: battery info based on REAM
    battery = set(Filter(technology,['Battery']))
    gen_project_info.loc[gen_project_info['technology'].isin(battery), 'gen_storage_efficiency'] = 0.75
    gen_project_info['gen_storage_efficiency'] = gen_project_info['gen_storage_efficiency'].fillna('.')
    gen_project_info.loc[gen_project_info['technology'].isin(battery), 'gen_store_to_release_ratio'] = 1
    gen_project_info['gen_store_to_release_ratio'] = gen_project_info['gen_store_to_release_ratio'].fillna('.')

    # based on manually created dictionaries
    gen_project_info['gen_energy_source'] = gen_project_info['technology'].apply(lambda x: energy_tech[x])
    gen_project_info['gen_is_cogen'] = gen_project_info['technology'].apply(lambda x: cogen_tech[x])
    gen_project_info['gen_is_baseload'] = gen_project_info['technology'].apply(lambda x: baseload_tech[x])
    gen_project_info['gen_scheduled_outage_rate'] = gen_project_info['technology'].apply(lambda x: sched_outage_tech[x])
    gen_project_info['gen_forced_outage_rate'] = gen_project_info['technology'].apply(lambda x: forced_outage_tech[x])

    # additional columns based on REAM
    gen_project_info['gen_min_build_capacity'] = 0 # REAM is just 0 or .
    gen_project_info['gen_can_provide_cap_reserves'] = 1 # all ones in scenario 178. either 1 or 0

    # these are blanks in scenario 178
    gen_project_info['gen_self_discharge_rate'] = '.'
    gen_project_info['gen_discharge_efficiency'] = '.'
    gen_project_info['gen_land_use_rate'] = '.'
    gen_project_info['gen_storage_energy_to_power_ratio'] = '.'
    

    # retirement ages based on settings file still need to be updated
    gen_project_info['gen_max_age'] = gen_project_info['technology'].apply(lambda x: retirement_age[x])
    
    # GENERATION_PROJECT - the all_gen.index column has NaNs for the new generators.  Use actual index for all_gen
    gen_project_info['GENERATION_PROJECT'] = gen_project_info.index +1
    gen_project_info['gen_dbid'] = gen_project_info['GENERATION_PROJECT']
    # rename columns
    gen_project_info.rename(columns = {'technology': 'gen_tech', 'region': 'gen_load_zone', 
                        'Heat_Rate_MMBTU_per_MWh': 'gen_full_load_heat_rate', 'Var_OM_Cost_per_MWh': 'gen_variable_om',
                        'Existing_Cap_MW': 'gen_capacity_limit_mw'}, inplace = True) #'index':'GENERATION_PROJECT',
    
    # drop heat_load_shifting (not in SWITCH)
    gen_project_info.drop(gen_project_info[gen_project_info['gen_tech'] == 'heat_load_shifting'].index, inplace = True)
    
    cols = ['GENERATION_PROJECT','gen_tech', 'gen_energy_source', 'gen_load_zone', 'gen_max_age', 
            'gen_is_variable', 'gen_is_baseload', 'gen_full_load_heat_rate', 'gen_variable_om', 
            'gen_connect_cost_per_mw', 'gen_dbid', 'gen_scheduled_outage_rate', 'gen_forced_outage_rate', 
            'gen_capacity_limit_mw', 'gen_min_build_capacity', 'gen_is_cogen', 'gen_storage_efficiency',
            'gen_store_to_release_ratio', 'gen_can_provide_cap_reserves', 'gen_self_discharge_rate', 
            'gen_discharge_efficiency', 'gen_land_use_rate', 'gen_storage_energy_to_power_ratio'] # index
    
    # remove NaN
    gen_project_info['gen_variable_om'] = gen_project_info['gen_variable_om'].fillna(0) 
    gen_project_info['gen_connect_cost_per_mw'] = gen_project_info['gen_variable_om'].fillna(0)
    gen_project_info['gen_capacity_limit_mw'] = gen_project_info['gen_variable_om'].fillna('.')
    
    gen_project_info['gen_full_load_heat_rate'] = gen_project_info['gen_full_load_heat_rate'].replace(0,'.')
    
    
    gen_project_info = gen_project_info[cols]
    return gen_project_info

In [6]:
gen_project_info = generation_projects_info(all_gen, spur_capex_mw_mile, retirement_age, cogen_tech, baseload_tech, 
                            energy_tech, sched_outage_tech, forced_outage_tech)
gen_project_info

,GENERATION_PROJECT,gen_tech,gen_energy_source,gen_load_zone,gen_max_age,gen_is_variable,gen_is_baseload,gen_full_load_heat_rate,gen_variable_om,gen_connect_cost_per_mw,...,gen_capacity_limit_mw,gen_min_build_capacity,gen_is_cogen,gen_storage_efficiency,gen_store_to_release_ratio,gen_can_provide_cap_reserves,gen_self_discharge_rate,gen_discharge_efficiency,gen_land_use_rate,gen_storage_energy_to_power_ratio
0,1,Biomass,Bio Solid,WECC_AZ,50,False,False,9210.0,3.270,3.270,...,3.270,0,False,.,.,1,.,.,.,.
1,2,Biomass,Bio Solid,WECC_AZ,50,False,False,16526.0,5.867,5.867,...,5.867,0,False,.,.,1,.,.,.,.
2,3,Biomass,Bio Solid,WECC_AZ,50,False,False,11013.0,3.910,3.910,...,3.910,0,False,.,.,1,.,.,.,.
3,4,Biomass,Bio Solid,WECC_AZ,50,False,False,11013.0,3.910,3.910,...,3.910,0,False,.,.,1,.,.,.,.
4,5,Conventional Hydroelectric,Water,WECC_AZ,100,False,False,8773.0,0.000,0.000,...,0.000,0,False,.,.,1,.,.,.,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4428,4429,NaturalGas_CCCCSAvgCF_Conservative,Gas,WECC_WY,60,False,False,7159.0,5.494,5.494,...,5.494,0,False,.,.,1,.,.,.,.
4429,4430,NaturalGas_CCAvgCF_Moderate,Gas,WECC_WY,60,False,False,6363.0,1.668,1.668,...,1.668,0,False,.,.,1,.,.,.,.
4430,4431,NaturalGas_CTAvgCF_Moderate,Gas,WECC_WY,50,False,False,9717.0,4.736,4.736,...,4.736,0,False,.,.,1,.,.,.,.
4431,4432,Battery_*_Moderate,Electricity,WECC_WY,15,False,False,.,0.000,0.000,...,0.000,0,False,0.75,1.0,1,.,.,.,.


In [7]:
# checking for heat_load_shifting
gen_project_info.loc[gen_project_info['gen_energy_source']==False]

,GENERATION_PROJECT,gen_tech,gen_energy_source,gen_load_zone,gen_max_age,gen_is_variable,gen_is_baseload,gen_full_load_heat_rate,gen_variable_om,gen_connect_cost_per_mw,...,gen_capacity_limit_mw,gen_min_build_capacity,gen_is_cogen,gen_storage_efficiency,gen_store_to_release_ratio,gen_can_provide_cap_reserves,gen_self_discharge_rate,gen_discharge_efficiency,gen_land_use_rate,gen_storage_energy_to_power_ratio


In [8]:
# drop retired plants
# information based on gen_build_predetermined notebook
gen_project_info.drop(gen_project_info[gen_project_info['GENERATION_PROJECT'].isin(
                [3225, 4070])].index, inplace = True)
gen_project_info

,GENERATION_PROJECT,gen_tech,gen_energy_source,gen_load_zone,gen_max_age,gen_is_variable,gen_is_baseload,gen_full_load_heat_rate,gen_variable_om,gen_connect_cost_per_mw,...,gen_capacity_limit_mw,gen_min_build_capacity,gen_is_cogen,gen_storage_efficiency,gen_store_to_release_ratio,gen_can_provide_cap_reserves,gen_self_discharge_rate,gen_discharge_efficiency,gen_land_use_rate,gen_storage_energy_to_power_ratio
0,1,Biomass,Bio Solid,WECC_AZ,50,False,False,9210.0,3.270,3.270,...,3.270,0,False,.,.,1,.,.,.,.
1,2,Biomass,Bio Solid,WECC_AZ,50,False,False,16526.0,5.867,5.867,...,5.867,0,False,.,.,1,.,.,.,.
2,3,Biomass,Bio Solid,WECC_AZ,50,False,False,11013.0,3.910,3.910,...,3.910,0,False,.,.,1,.,.,.,.
3,4,Biomass,Bio Solid,WECC_AZ,50,False,False,11013.0,3.910,3.910,...,3.910,0,False,.,.,1,.,.,.,.
4,5,Conventional Hydroelectric,Water,WECC_AZ,100,False,False,8773.0,0.000,0.000,...,0.000,0,False,.,.,1,.,.,.,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4428,4429,NaturalGas_CCCCSAvgCF_Conservative,Gas,WECC_WY,60,False,False,7159.0,5.494,5.494,...,5.494,0,False,.,.,1,.,.,.,.
4429,4430,NaturalGas_CCAvgCF_Moderate,Gas,WECC_WY,60,False,False,6363.0,1.668,1.668,...,1.668,0,False,.,.,1,.,.,.,.
4430,4431,NaturalGas_CTAvgCF_Moderate,Gas,WECC_WY,50,False,False,9717.0,4.736,4.736,...,4.736,0,False,.,.,1,.,.,.,.
4431,4432,Battery_*_Moderate,Electricity,WECC_WY,15,False,False,.,0.000,0.000,...,0.000,0,False,0.75,1.0,1,.,.,.,.


# graph_tech_colors

In [9]:
graph_tech_colors_data = {'gen_type':['Biomass','Coal','Gas','Geothermal','Hydro','Nuclear','Oil','Solar',
                            'Storage','Waste','Wave','Wind','Other'],'color':['green','saddlebrown','gray',
                            'red','royalblue','blueviolet','orange','gold','aquamarine','black','blue',
                                                                              'deepskyblue','white']}
graph_tech_colors_table = pd.DataFrame(graph_tech_colors_data)
graph_tech_colors_table.insert(0,'map_name','default')
graph_tech_colors_table

,map_name,gen_type,color
0,default,Biomass,green
1,default,Coal,saddlebrown
2,default,Gas,gray
3,default,Geothermal,red
4,default,Hydro,royalblue
5,default,Nuclear,blueviolet
6,default,Oil,orange
7,default,Solar,gold
8,default,Storage,aquamarine
9,default,Waste,black


# graph_tech_types

In [10]:
gen_type_tech = {'Onshore Wind Turbine':'Wind', 'Biomass': 'Biomass', 'Conventional Hydroelectric':'Hydro', 
                'Conventional Steam Coal': 'Coal', 'Natural Gas Fired Combined Cycle':'Gas', 
                 'Natural Gas Fired Combustion Turbine':'Gas', 'Natural Gas Steam Turbine':'Gas', 
                'Nuclear':'Nuclear', 'Solar Photovoltaic':'Solar', 'Hydroelectric Pumped Storage':'Hydro', 
                'Offshore Wind Turbine':'Wind', 'NaturalGas_CCCCSAvgCF_Conservative':'Gas', 
                'NaturalGas_CCAvgCF_Moderate':'Gas', 'NaturalGas_CTAvgCF_Moderate':'Gas', 
                 'Battery_*_Moderate':'Storage', 'NaturalGas_CCS100_Moderate':'Gas'}

In [11]:
gen_tech = gen_project_info['gen_tech'].unique()
graph_tech_types_table = pd.DataFrame(columns=['map_name', 'gen_type', 'gen_tech', 'energy_source'])
graph_tech_types_table['gen_tech'] = gen_tech
graph_tech_types_table['energy_source'] = graph_tech_types_table['gen_tech'].apply(lambda x: energy_tech[x])
graph_tech_types_table['map_name'] = 'default'
graph_tech_types_table['gen_type'] = graph_tech_types_table['gen_tech'].apply(lambda x: gen_type_tech[x])
graph_tech_types_table

,map_name,gen_type,gen_tech,energy_source
0,default,Biomass,Biomass,Bio Solid
1,default,Hydro,Conventional Hydroelectric,Water
2,default,Coal,Conventional Steam Coal,Coal
3,default,Hydro,Hydroelectric Pumped Storage,Water
4,default,Gas,Natural Gas Fired Combined Cycle,Gas
5,default,Gas,Natural Gas Fired Combustion Turbine,Gas
6,default,Gas,Natural Gas Steam Turbine,Gas
7,default,Nuclear,Nuclear,Uranium
8,default,Wind,Onshore Wind Turbine,Wind
9,default,Solar,Solar Photovoltaic,Solar


# non_fuel_energy_sources

In [12]:
fuels = fuel_prices['fuel'].unique()
fuels = [fuel.capitalize() for fuel in fuels]
non_fuel_table = graph_tech_types_table[~graph_tech_types_table['energy_source'].isin(fuels)]
non_fuel_energy = list(set(non_fuel_table['energy_source'].to_list()))
non_fuel_energy_table = pd.DataFrame(non_fuel_energy, columns=['energy_source'])
non_fuel_energy_table

,energy_source
0,Bio Solid
1,Gas
2,Electricity
3,Water
4,Wind
5,Solar


# modify gen_projects_info based on non_fuel

In [13]:
gen_project_info['gen_full_load_heat_rate'] = gen_project_info.apply(
                lambda row: '.' if row.gen_energy_source in non_fuel_energy else row.gen_full_load_heat_rate,  axis=1)
gen_project_info

,GENERATION_PROJECT,gen_tech,gen_energy_source,gen_load_zone,gen_max_age,gen_is_variable,gen_is_baseload,gen_full_load_heat_rate,gen_variable_om,gen_connect_cost_per_mw,...,gen_capacity_limit_mw,gen_min_build_capacity,gen_is_cogen,gen_storage_efficiency,gen_store_to_release_ratio,gen_can_provide_cap_reserves,gen_self_discharge_rate,gen_discharge_efficiency,gen_land_use_rate,gen_storage_energy_to_power_ratio
0,1,Biomass,Bio Solid,WECC_AZ,50,False,False,.,3.270,3.270,...,3.270,0,False,.,.,1,.,.,.,.
1,2,Biomass,Bio Solid,WECC_AZ,50,False,False,.,5.867,5.867,...,5.867,0,False,.,.,1,.,.,.,.
2,3,Biomass,Bio Solid,WECC_AZ,50,False,False,.,3.910,3.910,...,3.910,0,False,.,.,1,.,.,.,.
3,4,Biomass,Bio Solid,WECC_AZ,50,False,False,.,3.910,3.910,...,3.910,0,False,.,.,1,.,.,.,.
4,5,Conventional Hydroelectric,Water,WECC_AZ,100,False,False,.,0.000,0.000,...,0.000,0,False,.,.,1,.,.,.,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4428,4429,NaturalGas_CCCCSAvgCF_Conservative,Gas,WECC_WY,60,False,False,.,5.494,5.494,...,5.494,0,False,.,.,1,.,.,.,.
4429,4430,NaturalGas_CCAvgCF_Moderate,Gas,WECC_WY,60,False,False,.,1.668,1.668,...,1.668,0,False,.,.,1,.,.,.,.
4430,4431,NaturalGas_CTAvgCF_Moderate,Gas,WECC_WY,50,False,False,.,4.736,4.736,...,4.736,0,False,.,.,1,.,.,.,.
4431,4432,Battery_*_Moderate,Electricity,WECC_WY,15,False,False,.,0.000,0.000,...,0.000,0,False,0.75,1.0,1,.,.,.,.


In [15]:
gen_project_info.to_csv(r'SWITCH_Inputs\generation_projects_info.csv', index = False)
graph_tech_colors_table.to_csv(r'SWITCH_Inputs\graph_tech_colors.csv', index = False)
graph_tech_types_table.to_csv(r'SWITCH_Inputs\graph_tech_types.csv', index = False)
non_fuel_energy_table.to_csv(r'SWITCH_Inputs\non_fuel_energy_sources.csv', index = False)